<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/NasQ_TV_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
from tvDatafeed import TvDatafeed, Interval
import multiprocessing
import time
import glob

In [2]:
def get_nasq_syms():
  with open("watchlist_NASDAQ.txt", "r") as f:
    lines = f.readlines()
    f.close()
  line = lines[0].split(",")
  symbols = []
  for l in line:
    x = l.split(":")
    symbols.append(x[1])
  return symbols
def download_data_p(tv,sym):
    data =tv.get_hist(sym,exchange="binance", interval=Interval.in_15_minute, n_bars=50000)
    try:
        data.to_csv("data/{}.csv".format(sym))
    except:
      pass  
def download_data(symbols):
  work_with_dir()
  tv = TvDatafeed()
  for sym in symbols:
    print(symbols.index(sym)+1, "/", len(symbols))
    p = multiprocessing.Process(target=download_data_p, name="dd", args=(tv,sym,))
    p.start()
    time.sleep(2)
    p.terminate()
    p.join()

In [61]:
download_data(["BTCUSDT","ETHUSDT","XRPUSDT","TRXUSDT"])
#download_data(get_nasq_syms())

you are using nologin method, data you access may be limited


Data Folder Removed
1 / 4
2 / 4
3 / 4
4 / 4


In [12]:
clmns = [
    "Open 1-2", "Open 1-3", "Open 1-4", "Open 1-5", "Open 1-6", "Open 1-7", "Open 1-8", "Open 1-9", "Open 1-10", "Open 1-11", "Open 1-12", "Open 1-13", "Open 1-14", "Open 1-15", "Open 1-16",
    "High 1-2", "High 1-3", "High 1-4", "High 1-5", "High 1-6", "High 1-7", "High 1-8", "High 1-9", "High 1-10", "High 1-11", "High 1-12", "High 1-13", "High 1-14", "High 1-15", "High 1-16",
    "Low 1-2", "Low 1-3", "Low 1-4", "Low 1-5", "Low 1-6", "Low 1-7", "Low 1-8", "Low 1-9", "Low 1-10", "Low 1-11", "Low 1-12", "Low 1-13", "Low 1-14", "Low 1-15", "Low 1-16",
    "Close 1-2", "Close 1-3", "Close 1-4", "Close 1-5", "Close 1-6", "Close 1-7", "Close 1-8", "Close 1-9", "Close 1-10", "Close 1-11", "Close 1-12", "Close 1-13", "Close 1-14", "Close 1-15", "Close 1-16",
    "Volume 1-2", "Volume 1-3", "Volume 1-4", "Volume 1-5", "Volume 1-6", "Volume 1-7", "Volume 1-8", "Volume 1-9", "Volume 1-10", "Volume 1-11", "Volume 1-12", "Volume 1-13", "Volume 1-14", "Volume 1-15", "Volume 1-16",
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if "symbol" in data.columns:
        data.drop("symbol", axis=1, inplace=True)  
    if "Adj Close" in data.columns:
        data.drop("Adj Close", axis=1, inplace=True)
    if "datetime" in data.columns:
        data.drop("datetime",axis=1,inplace=True)
    if len(pd.DataFrame(data).columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)  
    llst = [0, 1, 2, 3, 4]

    for i in range(16, data.shape[0]):
        grow = []
        srow = []

        for j in llst:
          srow.append([
                data[i-1][j] - data[i-2][j],
                data[i-2][j] - data[i-3][j],
                data[i-3][j] - data[i-4][j],
                data[i-4][j] - data[i-5][j],
                data[i-5][j] - data[i-6][j],
                data[i-6][j] - data[i-7][j],
                data[i-7][j] - data[i-8][j],
                data[i-8][j] - data[i-9][j],
                data[i-9][j] - data[i-10][j],
                data[i-10][j] - data[i-11][j],
                data[i-11][j] - data[i-12][j],
                data[i-12][j] - data[i-13][j],
                data[i-13][j] - data[i-14][j],
                data[i-14][j] - data[i-15][j],
                data[i-15][j] - data[i-16][j],
           ])
        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))


        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"
        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [4]:
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     data = np.array(unattached_dfs[0])
     for i in unattached_dfs[1:]:
           data = np.append(data, np.array(i), axis=0)

     unattached_dfs = []
     
     data = pd.DataFrame(data, columns=clmns)
 

     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  print("Done")
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [13]:
folder_name = extract_data(1)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1
Processing File:
Done
1  
(4984, 77)
(3987, 75) (3987, 2)
(997, 75) (997, 2)


In [10]:
model = Sequential()



model.add(Dense(128, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))



opt = tf.keras.optimizers.Adamax()

model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               9728      
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 128)               16512     
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dense_9 (Dense)             (None, 2)                 258       
                                                                 
Total params: 59,522
Trainable params: 59,522
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(xTrain,yTrain,epochs=30,batch_size=16,validation_data=(xTest,yTest))

Epoch 1/30
250/250 [==============================] - 2s 4ms/step - loss: 0.6961 - accuracy: 0.5036 - val_loss: 0.6927 - val_accuracy: 0.5055
Epoch 2/30
250/250 [==============================] - 1s 3ms/step - loss: 0.6915 - accuracy: 0.5275 - val_loss: 0.6931 - val_accuracy: 0.5145
Epoch 3/30
250/250 [==============================] - 1s 3ms/step - loss: 0.6887 - accuracy: 0.5332 - val_loss: 0.6942 - val_accuracy: 0.5206
Epoch 4/30
250/250 [==============================] - 1s 3ms/step - loss: 0.6849 - accuracy: 0.5475 - val_loss: 0.7003 - val_accuracy: 0.5075
Epoch 5/30
250/250 [==============================] - 1s 3ms/step - loss: 0.6787 - accuracy: 0.5638 - val_loss: 0.7013 - val_accuracy: 0.5216
Epoch 6/30
250/250 [==============================] - 1s 3ms/step - loss: 0.6732 - accuracy: 0.5834 - val_loss: 0.6982 - val_accuracy: 0.5186
Epoch 7/30
250/250 [==============================] - 1s 3ms/step - loss: 0.6642 - accuracy: 0.5924 - val_loss: 0.7057 - val_accuracy: 0.5206
Epoch 